In [1]:
import zipfile
import os

# Percorso al file zip
zip_path = "./dataset.zip"

# Percorso di estrazione (stessa cartella, sottocartella 'dataset/')
extract_path = "./dataset/"

# Estrai solo se la cartella non esiste
if not os.path.exists(extract_path):
    print("Estrazione in corso...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Estrazione completata.")
else:
    print("La cartella 'dataset/' esiste già. Estrazione saltata.")


La cartella 'dataset/' esiste già. Estrazione saltata.


# **Analisi del dataset: distribuzione delle classi**

Questa sezione esegue una prima analisi esplorativa sul dataset. Dopo aver aggiunto il percorso personalizzato al `sys.path` (per poter importare moduli personalizzati dal progetto), viene utilizzata la funzione `analyze_class_distribution()` da `analysis_utils`.
Questa funzione esamina le maschere di segmentazione all’interno del dataset per calcolare:

*   La percentuale di pixel occupata da ciascuna classe
*   La presenza di ciascuna classe in termini di immagini in cui appare

Il risultato è un `DataFrame` Pandas con una rappresentazione tabellare della distribuzione di tutte le 9 classi, visualizzato con formattazione.

In [2]:
#ANALISI DATASET, RESTITUISCE UN DATA
import sys
import pandas as pd

# Importa la funzione di analisi
from utils.analysis_utils import analyze_class_distribution

# Specifica il percorso del dataset
DATASET_DIR = './dataset/dataset/'

# Esegui l'analisi
df = analyze_class_distribution(DATASET_DIR)

# Mostra il DataFrame in forma tabellare
df.style.set_caption("Distribuzione delle Classi").format(precision=2)


Analyzing: 100%|██████████| 932/932 [00:09<00:00, 99.15it/s] 


,Class,Pixel %,Presence % (images)
0,0,2.86,100.00
1,1,15.04,58.97
2,2,13.36,68.96
3,3,15.20,55.85
4,4,0.18,4.51
5,5,0.87,17.62
6,6,6.24,38.13
7,7,36.06,99.14
8,8,10.19,90.66


# **Suddivisione del dataset in Train/Val e Test con bilanciamento delle classi**

### 📦 Suddivisione del Dataset in Train/Val e Test

Questa sezione gestisce la **suddivisione del dataset in due parti principali**:
- **Train/Validation (90%)**
- **Test set (10%)**

L'obiettivo è ottenere un **test set rappresentativo**, con una distribuzione delle classi simile a quella dell'intero dataset. Lo split viene fatto in modo **greedy** per bilanciare:
- La distribuzione dei **pixel per classe**
- La presenza/assenza delle classi nelle immagini

---

#### ⚙️ Importazione dei moduli

Vengono caricati:
- Moduli di sistema (`os`, `sys`, `numpy`, `pandas`, ecc.)
- Librerie custom del progetto (`class_mapping`, `split_utils`, `analysis_utils`)

---

#### 📁 Definizione dei percorsi

- `PROJECT_DIR`: percorso principale del progetto
- `DATASET_DIR`: contiene le cartelle delle immagini
- `train_val_split.txt` e `test_split.txt`: file `.txt` con l’elenco dei campioni suddivisi

---

#### 🔐 Controllo esistenza file

Per evitare di ripetere la suddivisione, il codice controlla se i file di split esistono già. Se sì, il processo termina subito.

---

#### 📊 Analisi delle immagini

Attraverso la funzione `analyze_images_distribution()` si ottengono per ciascuna immagine:
- La **distribuzione percentuale di pixel per classe**
- La **presenza binaria** (classe presente/sì-no)

Inoltre, viene calcolata la distribuzione **globale** dell'intero subset.

---

#### 🔀 Algoritmo di Split Greedy

La funzione `greedy_split()` esegue lo split mantenendo:
- Proporzioni simili per classe (pixel)
- Presenza equilibrata delle classi nel test set

Il parametro `test_ratio = 0.10` definisce il 10% per il test.

---

#### 💾 Salvataggio dei risultati

I nomi delle immagini vengono scritti in:
- `test_split.txt`: campioni del test
- `train_val_split.txt`: campioni usati per training + validation

---

#### 📈 Confronto tra distribuzioni

Infine, viene generata una tabella `pandas` con il confronto tra:
- **Distribuzione di pixel globale** vs test
- **Presenza delle classi globale** vs test

Questa tabella verifica la **coerenza statistica dello split**.


In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from collections import defaultdict

from utils.split_utils import (
    compute_distribution,
    compute_presence_distribution,
    greedy_split
)
from utils.analysis_utils import analyze_images_distribution


def aggregate_counts(list_of_dicts):
    """Somma una lista di dizionari {classe: valore} in un unico dizionario aggregato."""
    total = defaultdict(int)
    for d in list_of_dicts:
        for k, v in d.items():
            total[k] += v
    return total

def convert_set_to_dict(class_set):
    """Converte un set come {1, 2} in {1: 1, 2: 1}"""
    return {cls: 1 for cls in class_set}


# === Percorsi ===
PROJECT_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14"
DATASET_DIR = os.path.join(PROJECT_DIR, "dataset/dataset")
TRAINVAL_FILE = os.path.join(PROJECT_DIR, "train_val_split.txt")
TEST_FILE = os.path.join(PROJECT_DIR, "test_split.txt")

# === Controllo esistenza split ===
split_exists = os.path.exists(TRAINVAL_FILE) and os.path.exists(TEST_FILE)

# === Caricamento split se esistono ===
if split_exists:
    print("I file di split esistono già. Lo split non verrà rieseguito.")
    with open(TRAINVAL_FILE, "r") as f:
        trainval_set = [line.strip() for line in f.readlines()]
    with open(TEST_FILE, "r") as f:
        test_set = [line.strip() for line in f.readlines()]
else:
    # === Carica elenco immagini completo da usare nello split ===
    with open(TRAINVAL_FILE, "r") as f:
        full_image_list = [line.strip() for line in f.readlines()]

# === Analisi immagini (su tutto il dataset) ===
(
    image_pixel_distributions,
    image_class_presence,
    global_pixel_dist,
    global_presence_dist,
    _  # immagini analizzate (non necessario qui)
) = analyze_images_distribution(DATASET_DIR)

# === Ricava CLASS_NAMES dinamicamente ===
CLASS_NAMES = sorted(set(global_pixel_dist.keys()) | set(global_presence_dist.keys()))

# === Se lo split non esiste, esegui split greedy ===
if not split_exists:
    print("Split non trovato: eseguo il greedy split.")
    test_ratio = 0.10
    test_set, trainval_set, test_pixel_counts, test_class_presence = greedy_split(
        image_pixel_distributions,
        image_class_presence,
        global_pixel_dist,
        global_presence_dist,
        CLASS_NAMES,
        test_ratio
    )

    # === Salvataggio .txt ===
    with open(TEST_FILE, "w") as f:
        for name in test_set:
            f.write(name + "\n")
    with open(TRAINVAL_FILE, "w") as f:
        for name in trainval_set:
            f.write(name + "\n")
else:
    # === Recupera conteggi da immagini ===
    test_pixel_counts = [image_pixel_distributions[n] for n in test_set]
    test_class_presence = [image_class_presence[n] for n in test_set]

# === Recupera anche trainval counts ===
trainval_pixel_counts = [image_pixel_distributions[n] for n in trainval_set]
trainval_class_presence = [image_class_presence[n] for n in trainval_set]

# === Converti set in dizionari per aggregazione presenza ===
test_presence_dicts = [convert_set_to_dict(s) for s in test_class_presence]
trainval_presence_dicts = [convert_set_to_dict(s) for s in trainval_class_presence]

# === Aggrega ===
agg_test_pixel = aggregate_counts(test_pixel_counts)
agg_trainval_pixel = aggregate_counts(trainval_pixel_counts)
agg_test_presence = aggregate_counts(test_presence_dicts)
agg_trainval_presence = aggregate_counts(trainval_presence_dicts)

# === Calcola distribuzioni ===
test_pixel_dist = compute_distribution(agg_test_pixel, CLASS_NAMES)
trainval_pixel_dist = compute_distribution(agg_trainval_pixel, CLASS_NAMES)
test_presence_dist = compute_presence_distribution(agg_test_presence, len(test_set), CLASS_NAMES)
trainval_presence_dist = compute_presence_distribution(agg_trainval_presence, len(trainval_set), CLASS_NAMES)

# === Confronto in DataFrame ===
df = pd.DataFrame({
    "Classe": CLASS_NAMES,
    "Pixel TrainVal (%)": [round(trainval_pixel_dist.get(c, 0)*100, 2) for c in CLASS_NAMES],
    "Pixel Test (%)":     [round(test_pixel_dist.get(c, 0)*100, 2) for c in CLASS_NAMES],
    "Presenza TrainVal (%)": [round(trainval_presence_dist.get(c, 0)*100, 2) for c in CLASS_NAMES],
    "Presenza Test (%)":     [round(test_presence_dist.get(c, 0)*100, 2) for c in CLASS_NAMES],
})

print("\nConfronto distribuzioni TrainVal vs Test:")
print(df)


I file di split esistono già. Lo split non verrà rieseguito.
Analisi immagini...


100%|██████████| 932/932 [00:08<00:00, 115.59it/s]


Confronto distribuzioni TrainVal vs Test:
   Classe  Pixel TrainVal (%)  Pixel Test (%)  Presenza TrainVal (%)  \
0       0                2.85            2.94                 100.00   
1       1               15.04           15.06                  58.95   
2       2               13.35           13.47                  68.97   
3       3               15.19           15.29                  55.85   
4       4                0.19            0.10                   4.53   
5       5                0.86            0.87                  17.66   
6       6                6.24            6.24                  38.07   
7       7               36.06           36.10                  99.16   
8       8               10.22            9.93                  90.69   

   Presenza Test (%)  
0             100.00  
1              59.14  
2              68.82  
3              55.91  
4               4.30  
5              17.20  
6              38.71  
7              98.92  
8              90.32  


# **Implementazione della K-Fold Cross-Validation sul Training Set**
### 📚 Suddivisione K-Fold del Dataset (Train/Val)

Questa sezione realizza la **suddivisione del dataset train/val (90% dei dati)** in **K fold** per eseguire una **cross-validation stratificata**.  
Ogni fold verrà salvato in un file `.txt` e potrà essere usato per addestrare e validare il modello su diverse porzioni del dataset, migliorando la generalizzazione.

---

#### ⚙️ Importazioni e configurazioni

Vengono importati:
- Moduli standard (`os`, `numpy`, `pandas`, ecc.)
- Moduli custom del progetto (`class_mapping`, `split_utils`, `analysis_utils`)
- La classe `KFold` da `sklearn.model_selection` per generare gli indici

Sono inoltre definiti i percorsi:
- `PROJECT_DIR`, `DATASET_DIR`: path base e dataset
- `SPLIT_DIR`, `TRAIN_DIR`, `VAL_DIR`: directory in cui salvare gli split
- `TRAINVAL_FILE`: file `.txt` con le immagini da dividere (generate precedentemente)

---

#### 📄 Caricamento e analisi delle immagini

Dalla lista `train_val_split.txt`, si caricano i nomi delle immagini e si esegue un’**analisi statistica**:
- `image_pixel_distributions`: percentuale di pixel per classe, per immagine
- `image_class_presence`: classi presenti per immagine (sì/no)
- `global_pixel_dist` e `global_presence_dist`: valori aggregati sull’intero set
- `images_list`: lista effettiva delle immagini analizzate

Queste metriche potranno essere usate per valutare la qualità dello split (anche se non direttamente in questa cella).

---

#### 🔀 K-Fold Cross-Validation (K=5)

Utilizzando `KFold` con `shuffle=True` e `random_state=42`, vengono generati 5 fold con indici diversi per ogni split. Per ogni fold:
- Si costruisce il `train_set` e il `val_set`
- Si salvano due file:
  - `train_foldN.txt` → immagini per l'addestramento
  - `val_foldN.txt` → immagini per la validazione

Questi file verranno usati nel training loop per caricare dinamicamente i dati in base al fold.

---

#### 💾 Output


In [4]:
import os
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
from collections import defaultdict

from utils.split_utils import compute_distribution, compute_presence_distribution, compute_distance
from utils.analysis_utils import analyze_images_distribution

# === CONFIG ===
K = 5
PROJECT_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14"
DATASET_DIR = os.path.join(PROJECT_DIR, "dataset/dataset")
SPLIT_DIR = os.path.join(PROJECT_DIR, "kfold_splits")
TRAINVAL_FILE = os.path.join(PROJECT_DIR, "train_val_split.txt")
TRAIN_DIR = os.path.join(SPLIT_DIR, "train")
VAL_DIR = os.path.join(SPLIT_DIR, "val")

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

# === Carica elenco immagini da usare nello split ===
with open(TRAINVAL_FILE, "r") as f:
    images_subset = [line.strip() for line in f.readlines()]

# === Analisi immagini solo su train_val_split.txt ===
(
    image_pixel_distributions,
    image_class_presence,
    global_pixel_dist,
    global_presence_dist,
    images_list
) = analyze_images_distribution(DATASET_DIR, subset=images_subset)
total_images = len(images_list)

# === K-FOLD SPLIT ===
kf = KFold(n_splits=K, shuffle=True, random_state=42)
fold = 1

for train_indices, val_indices in kf.split(images_list):
    train_path = os.path.join(TRAIN_DIR, f"train_fold{fold}.txt")
    val_path = os.path.join(VAL_DIR, f"val_fold{fold}.txt")

    # Salta fold se entrambi i file esistono già
    if os.path.exists(train_path) and os.path.exists(val_path):
        with open(train_path, "r") as f:
            train_count = len(f.readlines())
        with open(val_path, "r") as f:
            val_count = len(f.readlines())

        print(f"Fold {fold} già esistente: {train_count} train, {val_count} val")
        fold += 1
        continue

    train_set = [images_list[i] for i in train_indices]
    val_set = [images_list[i] for i in val_indices]

    # Salvataggio
    with open(train_path, "w") as f:
        for name in train_set:
            f.write(name + "\n")

    with open(val_path, "w") as f:
        for name in val_set:
            f.write(name + "\n")

    print(f"Fold {fold} salvato: {len(train_set)} train, {len(val_set)} val")
    fold += 1


Analisi immagini...


100%|██████████| 838/838 [00:07<00:00, 115.05it/s]

Fold 1 già esistente: 671 train, 167 val
Fold 2 già esistente: 671 train, 167 val
Fold 3 già esistente: 670 train, 168 val
Fold 4 già esistente: 670 train, 168 val
Fold 5 già esistente: 670 train, 168 val


#**Analisi delle Distribuzioni di Train e Validation nei K Fold**

Questa sezione verifica che la suddivisione in **train/val per ogni fold** della K-Fold cross-validation (K=10) mantenga una **distribuzione equilibrata delle classi** rispetto al dataset globale.

---

#### ⚙️ Import e configurazione

- Vengono importati moduli standard (`numpy`, `pandas`, `PIL`, ecc.) e moduli custom come `class_mapping`, `split_utils` e `analysis_utils`.
- Sono definiti i percorsi al dataset e alle directory dove sono salvati i file `.txt` generati nella fase di split precedente.

---

#### 📈 Analisi Globale

La funzione `analyze_images_distribution()` calcola:
- La **distribuzione dei pixel per classe** sull’intero dataset
- La **presenza percentuale** di ciascuna classe nelle immagini
Questa analisi serve come riferimento per confrontare i singoli fold.

---

#### 🔁 Funzione: `analyze_split_distribution()`

Questa funzione esegue la stessa analisi della distribuzione:
- Su una lista di immagini (`split_list`)
- Analizza il file `labels.png` per ciascuna cartella immagine
- Conta il numero di pixel per ogni classe (`pixel_counts`)
- Conta quante immagini contengono ogni classe (`presence_counts`)

Il tutto viene convertito in distribuzioni normalizzate:
- `pixel_dist` (% pixel per classe)
- `presence_dist` (% immagini che contengono ciascuna classe)

---

#### 🔍 Ciclo su tutti i fold

Per ciascun fold da 1 a K:
- Si leggono i file `train_foldN.txt` e `val_foldN.txt`
- Si analizzano train e validation separatamente con `analyze_split_distribution()`
- I risultati vengono confrontati con i valori globali

---

#### 📊 Output finale

Per ogni fold viene stampata una tabella `pandas` che permette di verificare se la **distribuzione di classi nei fold** è rappresentativa e bilanciata rispetto all’intero dataset.

---

📌 Questo passaggio è fondamentale per **validare la correttezza dello split**: se uno o più fold sono sbilanciati, le performance del modello potrebbero essere falsamente elevate o degradate.


In [5]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from collections import defaultdict
from tqdm import tqdm

from utils.split_utils import compute_distribution, compute_presence_distribution
from utils.analysis_utils import analyze_images_distribution

# === CONFIG ===
PROJECT_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14"
DATASET_DIR = os.path.join(PROJECT_DIR, "dataset/dataset")
SPLIT_DIR = os.path.join(PROJECT_DIR, "kfold_splits")
TRAIN_DIR = os.path.join(SPLIT_DIR, "train")
VAL_DIR = os.path.join(SPLIT_DIR, "val")
K = 5
NUM_CLASSES = 9
CLASS_NAMES = list(range(NUM_CLASSES))  # coerente con gli ID delle classi

# === Analisi globale con funzione centralizzata ===
print("Analisi distribuzione globale...")
_, _, global_pixel_dist, global_presence_dist, images_list = analyze_images_distribution(DATASET_DIR)
total_global = len(images_list)

# === Analisi di ogni fold ===
def analyze_split_distribution(split_list):
    pixel_counts = defaultdict(int)
    presence_counts = defaultdict(int)

    for folder in tqdm(split_list, desc="Analisi split"):
        label_path = os.path.join(DATASET_DIR, folder, "labels.png")
        if not os.path.isfile(label_path):
            continue

        label = np.array(Image.open(label_path), dtype=np.uint8).flatten()
        unique_classes = np.unique(label)

        for cls in unique_classes:
            count = np.sum(label == cls)
            if count > 0:
                pixel_counts[cls] += count
                presence_counts[cls] += 1

    total_images = len(split_list)
    pixel_dist = compute_distribution(pixel_counts, CLASS_NAMES)
    presence_dist = compute_presence_distribution(presence_counts, total_images, CLASS_NAMES)
    return pixel_dist, presence_dist

# === Analizza e confronta ogni fold ===
for fold in range(1, K + 1):
    print(f"\nFold {fold}")

    train_file = os.path.join(TRAIN_DIR, f"train_fold{fold}.txt")
    val_file = os.path.join(VAL_DIR, f"val_fold{fold}.txt")

    with open(train_file, "r") as f:
        train_list = [line.strip() for line in f.readlines()]
    with open(val_file, "r") as f:
        val_list = [line.strip() for line in f.readlines()]

    train_pixel_dist, train_presence_dist = analyze_split_distribution(train_list)
    val_pixel_dist, val_presence_dist = analyze_split_distribution(val_list)

    df = pd.DataFrame({
        "Classe": CLASS_NAMES,
        "Globale (Pixel%)": [round(global_pixel_dist[c] * 100, 2) for c in CLASS_NAMES],
        "Train (Pixel%)": [round(train_pixel_dist[c] * 100, 2) for c in CLASS_NAMES],
        "Val (Pixel%)": [round(val_pixel_dist[c] * 100, 2) for c in CLASS_NAMES],
        "Globale (Presenza%)": [round(global_presence_dist[c] * 100, 2) for c in CLASS_NAMES],
        "Train (Presenza%)": [round(train_presence_dist[c] * 100, 2) for c in CLASS_NAMES],
        "Val (Presenza%)": [round(val_presence_dist[c] * 100, 2) for c in CLASS_NAMES],
    })

    print(df.to_string(index=False))


Analisi distribuzione globale...
Analisi immagini...


100%|██████████| 932/932 [00:08<00:00, 104.64it/s]



Fold 1


Analisi split: 100%|██████████| 167/167 [00:01<00:00, 109.59it/s]


 Classe  Globale (Pixel%)  Train (Pixel%)  Val (Pixel%)  Globale (Presenza%)  Train (Presenza%)  Val (Presenza%)
      0              2.86            2.85          2.87               100.00             100.00           100.00
      1             15.04           15.07         14.92                58.97              59.02            58.68
      2             13.36           13.37         13.27                68.96              69.00            68.86
      3             15.20           15.21         15.09                55.85              55.89            55.69
      4              0.18            0.19          0.22                 4.51               4.47             4.79
      5              0.87            0.86          0.90                17.62              17.73            17.37
      6              6.24            6.21          6.37                38.13              38.00            38.32
      7             36.06           36.03         36.16                99.14              99.11 

...

 Classe  Globale (Pixel%)  Train (Pixel%)  Val (Pixel%)  Globale (Presenza%)  Train (Presenza%)  Val (Presenza%)
      0              2.86            2.86          2.82               100.00             100.00           100.00
      1             15.04           15.06         14.97                58.97              58.96            58.93
      2             13.36           13.33         13.43                68.96              68.96            69.05
      3             15.20           15.19         15.19                55.85              55.82            55.95
      4              0.18            0.19          0.19                 4.51               4.48             4.76
      5              0.87            0.86          0.87                17.62              17.61            17.86
      6              6.24            6.22          6.32                38.13              38.06            38.10
      7             36.06           36.06         36.04                99.14              99.10 

Analisi split: 100%|██████████| 168/168 [00:01<00:00, 114.98it/s]


 Classe  Globale (Pixel%)  Train (Pixel%)  Val (Pixel%)  Globale (Presenza%)  Train (Presenza%)  Val (Presenza%)
      0              2.86            2.85          2.84               100.00             100.00           100.00
      1             15.04           15.04         15.04                58.97              58.96            58.93
      2             13.36           13.35         13.34                68.96              68.96            69.05
      3             15.20           15.14         15.37                55.85              55.82            55.95
      4              0.18            0.19          0.19                 4.51               4.48             4.76
      5              0.87            0.86          0.88                17.62              17.61            17.86
      6              6.24            6.26          6.17                38.13              38.06            38.10
      7             36.06           36.06         36.05                99.14              99.10 

Analisi split: 100%|██████████| 168/168 [00:01<00:00, 91.12it/s] 

 Classe  Globale (Pixel%)  Train (Pixel%)  Val (Pixel%)  Globale (Presenza%)  Train (Presenza%)  Val (Presenza%)
      0              2.86            2.86          2.81               100.00             100.00           100.00
      1             15.04           14.99         15.23                58.97              58.96            58.93
      2             13.36           13.33         13.40                68.96              68.96            69.05
      3             15.20           15.18         15.20                55.85              55.82            55.95
      4              0.18            0.20          0.18                 4.51               4.63             4.17
      5              0.87            0.87          0.85                17.62              17.76            17.26
      6              6.24            6.27          6.13                38.13              38.21            37.50
      7             36.06           36.10         35.89                99.14              99.40 

# **Addestramento del modello DeepLabV3 + MobileNetV3 con K-Fold**

Questa sezione esegue il training del modello DeepLabV3 con backbone MobileNetV3 utilizzando la **K-Fold Cross Validation**.

---

#### ⚙️ Setup e Modello

- Inizializzazione del modello `DeepLabV3` con pesi preaddestrati
- Definizione della **loss combinata** (`CombinedLoss`), composta da Jaccard loss e Cross Entropy
- Configurazione dell’ottimizzatore e learning rate scheduler

---

#### 🔁 Ciclo di Addestramento per ciascun Fold

Per ogni fold:

- Creazione dei `DataLoader` per i subset `train` e `val`
- Training per un numero massimo di epoche (con early stopping)
- Calcolo delle metriche: **loss, accuracy e mIoU**
- Salvataggio del **miglior modello** in base alla mIoU
- Monitoraggio della **memoria GPU** utilizzata ad ogni epoca

---

#### 💾 Output

Al termine di ogni fold viene salvato:

- Il modello (`.pt`) migliore
- I valori delle metriche per ogni epoca
- Eventuali log di debug e monitoraggio GPU

---

📌 Questa pipeline permette di **valutare la robustezza del modello** su diversi subset del dataset, fornendo una stima più realistica delle sue performance.

In [1]:
import os
import random
import gc
import json
import numpy as np
import torch

from torch.utils.data import DataLoader

# Imposta seed per riproducibilità
SEED = 14
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



from torch.utils.data import DataLoader
from utils.dataset import SegmentationDataset
from utils.model import DeepLabV3PlusWithMobileNet
from utils.losses import get_weighted_crossentropy_loss, combined_loss
from utils.metrics import compute_miou, compute_pixel_accuracy

# CONFIG
BATCH_SIZE = 4
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 9
EPOCHS = 50
ALPHA = 0.7
LR = 1e-4
MIN_DELTA = 1e-4
PATIENCE = 10
MAX_MEMORY_MB = 5120  # 5 GB

RESULTS = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for fold in [5]:
    print(f"\n Inizio Fold {fold}\n{'-'*40}")

    train_dataset = SegmentationDataset(
        root_dir="/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset",
        id_list_file=f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_fold{fold}.txt",
        image_size=IMAGE_SIZE,
    )
    val_dataset = SegmentationDataset(
        root_dir="/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset",
        id_list_file=f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/val/val_fold{fold}.txt",
        image_size=IMAGE_SIZE
    )

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    model = DeepLabV3PlusWithMobileNet(num_classes=NUM_CLASSES).to(device)
    ce_loss_fn = get_weighted_crossentropy_loss(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5
    )

    best_miou = -1.0
    best_val_loss = float('inf')
    epochs_no_improve = 0
    fold_results = []

    for epoch in range(1, EPOCHS + 1):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = combined_loss(outputs, labels, ce_loss_fn, alpha=ALPHA)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        total_miou = 0.0
        total_acc = 0.0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = combined_loss(outputs, labels, ce_loss_fn, alpha=ALPHA)
                val_loss += loss.item()
                total_miou += compute_miou(outputs, labels, num_classes=NUM_CLASSES, ignore_index=0)
                total_acc += compute_pixel_accuracy(outputs, labels, ignore_index=0)

        val_loss /= len(val_loader)
        avg_miou = total_miou / len(val_loader)
        scheduler.step(val_loss)
        avg_acc = total_acc / len(val_loader)

        fold_results.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "mIoU": avg_miou,
            "accuracy": avg_acc
        })

        print(f"[Fold {fold} | Epoch {epoch:02d}] "
              f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
              f"mIoU: {avg_miou:.4f} | Acc: {avg_acc:.4f}")

        if avg_miou > best_miou:
            best_miou = avg_miou
            epochs_no_improve = 0
            model_path = f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/best_model_fold{fold}.pth"
            torch.save(model.state_dict(), model_path)
            print(f"Salvataggio modello Fold {fold} | Nuova mIoU: {best_miou:.4f}")

        if epoch == 1 or val_loss < best_val_loss - MIN_DELTA:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"EARLY STOPPING FOLD {fold}: no val_loss improvement for {PATIENCE} epochs.")
            break


        if device.type == 'cuda':
            mem_allocated = torch.cuda.memory_allocated(device) / (1024 ** 2)
            if mem_allocated > MAX_MEMORY_MB:
                print(f"GPU MEMORY LIMIT EXCEEDED: {mem_allocated:.2f} MB")
                break

        torch.cuda.empty_cache()
        gc.collect()

    RESULTS.append({
        "fold": fold,
        "best_miou": best_miou,
        "epoch_results": fold_results
    })

# ✅ Salva i risultati totali in un file
import json
with open("/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_results.json", "w") as f:
    json.dump(RESULTS, f, indent=2)

print("\nCross-validation completata. Tutti i modelli e risultati salvati.")


 Inizio Fold 5
----------------------------------------
[Fold 5 | Epoch 01] Train Loss: 1.1939 | Val Loss: 0.9365 | mIoU: 0.3788 | Acc: 0.7315
Salvataggio modello Fold 5 | Nuova mIoU: 0.3788
[Fold 5 | Epoch 02] Train Loss: 0.9250 | Val Loss: 0.7682 | mIoU: 0.4775 | Acc: 0.7943
Salvataggio modello Fold 5 | Nuova mIoU: 0.4775
[Fold 5 | Epoch 03] Train Loss: 0.7992 | Val Loss: 0.7168 | mIoU: 0.5013 | Acc: 0.7974
Salvataggio modello Fold 5 | Nuova mIoU: 0.5013
[Fold 5 | Epoch 04] Train Loss: 0.7068 | Val Loss: 0.6117 | mIoU: 0.5156 | Acc: 0.8125
Salvataggio modello Fold 5 | Nuova mIoU: 0.5156
[Fold 5 | Epoch 05] Train Loss: 0.6465 | Val Loss: 0.6071 | mIoU: 0.5134 | Acc: 0.8103
[Fold 5 | Epoch 06] Train Loss: 0.5898 | Val Loss: 0.5891 | mIoU: 0.5178 | Acc: 0.8114
Salvataggio modello Fold 5 | Nuova mIoU: 0.5178
[Fold 5 | Epoch 07] Train Loss: 0.5508 | Val Loss: 0.5941 | mIoU: 0.4896 | Acc: 0.7866
[Fold 5 | Epoch 08] Train Loss: 0.4980 | Val Loss: 0.5345 | mIoU: 0.5181 | Acc: 0.8142
Salvata

In [ ]:
import sys
sys.path.append("/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14")

import torch
import gc
from torch.utils.data import DataLoader
from utils.dataset import SegmentationDataset
from utils.model import DeepLabV3PlusWithMobileNet
from utils.losses import get_weighted_crossentropy_loss, combined_loss
from utils.metrics import compute_miou, compute_pixel_accuracy

# CONFIG
BATCH_SIZE = 4
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 9
EPOCHS = 20
ALPHA = 0.7
LR = 1e-4
PATIENCE = 10
MAX_MEMORY_MB = 5120  # 5 GB

# Dataset
train_dataset = SegmentationDataset(
    root_dir='/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset',
    id_list_file='/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_fold1.txt',
    image_size=IMAGE_SIZE
)
val_dataset = SegmentationDataset(
    root_dir='/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset',
    id_list_file='/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/val/val_fold1.txt',
    image_size=IMAGE_SIZE
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Device, modello, ottimizzatore
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeepLabV3PlusWithMobileNet(num_classes=NUM_CLASSES).to(device)
ce_loss_fn = get_weighted_crossentropy_loss(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Early stopping + salvataggio su mIoU
best_miou = -1.0
epochs_no_improve = 0

for epoch in range(1, EPOCHS + 1):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = combined_loss(outputs, labels, ce_loss_fn, alpha=ALPHA)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # VALIDAZIONE
    model.eval()
    val_loss = 0.0
    total_miou = 0.0
    total_acc = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = combined_loss(outputs, labels, ce_loss_fn, alpha=ALPHA)
            val_loss += loss.item()
            total_miou += compute_miou(outputs, labels, num_classes=NUM_CLASSES, ignore_index=0)
            total_acc += compute_pixel_accuracy(outputs, labels, ignore_index=0)

    val_loss /= len(val_loader)
    avg_miou = total_miou / len(val_loader)
    avg_acc = total_acc / len(val_loader)

    print(f"Epoch {epoch:02d}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"mIoU: {avg_miou:.4f} | Acc: {avg_acc:.4f}")

    # SALVATAGGIO se migliora la mIoU
    if avg_miou > best_miou:
        best_miou = avg_miou
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_model.pt")
        print(f"MODELLO SALVATO: nuova mIoU migliore = {best_miou:.4f}")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"EARLY STOPPING ATTIVATO: nessun miglioramento per {PATIENCE} epoche consecutive.")
            break

    # MEMORY CHECK (solo se su CUDA)
    if device.type == 'cuda':
        mem_allocated = torch.cuda.memory_allocated(device) / (1024 ** 2)  # in MB
        if mem_allocated > MAX_MEMORY_MB:
            print(f"MEMORIA GPU SUPERATA: {mem_allocated:.2f} MB > {MAX_MEMORY_MB} MB")
            break

    # Cleanup memoria
    torch.cuda.empty_cache()
    gc.collect()


Epoch 01/20 | Train Loss: 1.2090 | Val Loss: 1.0285 | mIoU: 0.3656 | Acc: 0.7268
MODELLO SALVATO: nuova mIoU migliore = 0.3656
Epoch 02/20 | Train Loss: 0.9399 | Val Loss: 0.8520 | mIoU: 0.3857 | Acc: 0.7423
MODELLO SALVATO: nuova mIoU migliore = 0.3857
Epoch 03/20 | Train Loss: 0.8016 | Val Loss: 0.8063 | mIoU: 0.4191 | Acc: 0.7339
MODELLO SALVATO: nuova mIoU migliore = 0.4191
Epoch 04/20 | Train Loss: 0.7095 | Val Loss: 0.7153 | mIoU: 0.4439 | Acc: 0.7386
MODELLO SALVATO: nuova mIoU migliore = 0.4439
Epoch 05/20 | Train Loss: 0.6383 | Val Loss: 0.6961 | mIoU: 0.4661 | Acc: 0.7519
MODELLO SALVATO: nuova mIoU migliore = 0.4661
Epoch 06/20 | Train Loss: 0.5797 | Val Loss: 0.6402 | mIoU: 0.4705 | Acc: 0.7571
MODELLO SALVATO: nuova mIoU migliore = 0.4705


In [4]:
import os
import albumentations as A
from tqdm import tqdm
from PIL import Image
import numpy as np

# Percorsi
DATASET_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset"
TXT_PATH = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_fold5.txt"
OUTPUT_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/augmented_train_fold5"
NEW_TXT_PATH = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_augmented_fold5.txt"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Augmentazioni
image_only_transform = A.Compose([
    A.ColorJitter(p=0.5)
])
joint_transform = A.Compose([
    A.HorizontalFlip(p=0.8),
    A.RandomRotate90(p=0.3)
])

# Lista di nomi da salvare nel nuovo txt
all_sample_ids = []

# Campioni da augmentare
with open(TXT_PATH, 'r') as f:
    sample_ids = [line.strip() for line in f.readlines()]

for sample_id in tqdm(sample_ids, desc="Saving original + augment"):
    input_dir = os.path.join(DATASET_DIR, sample_id)
    img_path = os.path.join(input_dir, "rgb.jpg")
    lbl_path = os.path.join(input_dir, "labels.png")

    if not os.path.exists(img_path) or not os.path.exists(lbl_path):
        print(f"⚠️ File mancante: {sample_id}")
        continue

    image = np.array(Image.open(img_path).convert("RGB"))
    label = np.array(Image.open(lbl_path))  # già in ID, può restare come array intero

    # --- Originale ---
    orig_out_dir = os.path.join(OUTPUT_DIR, sample_id)
    os.makedirs(orig_out_dir, exist_ok=True)
    Image.fromarray(image).save(os.path.join(orig_out_dir, "rgb.jpg"))
    Image.fromarray(label.astype("uint8")).save(os.path.join(orig_out_dir, "labels.png"))
    all_sample_ids.append(sample_id)

    # --- Augmentato ---
    joint = joint_transform(image=image, mask=label)
    aug_image = joint["image"]
    aug_label = joint["mask"]
    aug_image = image_only_transform(image=aug_image)["image"]

    aug_sample_id = f"{sample_id}_aug"
    aug_out_dir = os.path.join(OUTPUT_DIR, aug_sample_id)
    os.makedirs(aug_out_dir, exist_ok=True)
    Image.fromarray(aug_image).save(os.path.join(aug_out_dir, "rgb.jpg"))
    Image.fromarray(aug_label.astype("uint8")).save(os.path.join(aug_out_dir, "labels.png"))
    all_sample_ids.append(aug_sample_id)

# Scrivi il nuovo file .txt
with open(NEW_TXT_PATH, 'w') as f:
    for sample_id in all_sample_ids:
        f.write(sample_id + "\n")

print(f"✅ Completato! Salvato nuovo file: {NEW_TXT_PATH}")

Saving original + augment: 100%|██████████| 670/670 [00:29<00:00, 22.56it/s]

✅ Completato! Salvato nuovo file: /Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_augmented_fold5.txt


In [9]:
import os
import albumentations as A
from tqdm import tqdm
from PIL import Image
import numpy as np

# Percorsi
DATASET_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset"
TXT_PATH = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_fold5.txt"
OUTPUT_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/augmented456_train_fold5"
NEW_TXT_PATH = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_augmented456_fold5.txt"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Augmentazioni
image_only_transform = A.Compose([
    A.ColorJitter(p=0.5)
])
joint_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.08, scale_limit=0.2, rotate_limit=15, border_mode=0, p=0.7),
])

image_only_transform = A.Compose([
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25, p=0.5)
])

# Classi da considerare per augment mirata
TARGET_CLASSES = {4, 5, 6}

# Lista di nomi da salvare nel nuovo txt
all_sample_ids = []

# Campioni da augmentare
with open(TXT_PATH, 'r') as f:
    sample_ids = [line.strip() for line in f.readlines()]

for sample_id in tqdm(sample_ids, desc="Saving original + selective augment"):
    input_dir = os.path.join(DATASET_DIR, sample_id)
    img_path = os.path.join(input_dir, "rgb.jpg")
    lbl_path = os.path.join(input_dir, "labels.png")

    if not os.path.exists(img_path) or not os.path.exists(lbl_path):
        print(f"⚠️ File mancante: {sample_id}")
        continue

    image = np.array(Image.open(img_path).convert("RGB"))
    label = np.array(Image.open(lbl_path))  # già in ID

    # --- Originale ---
    orig_out_dir = os.path.join(OUTPUT_DIR, sample_id)
    os.makedirs(orig_out_dir, exist_ok=True)
    Image.fromarray(image).save(os.path.join(orig_out_dir, "rgb.jpg"))
    Image.fromarray(label.astype("uint8")).save(os.path.join(orig_out_dir, "labels.png"))
    all_sample_ids.append(sample_id)

    # Verifica se contiene almeno una delle classi 4,5,6
    if not TARGET_CLASSES.intersection(np.unique(label)):
        continue  # Skip augment se non contiene le classi target

    # --- Augmentato ---
    joint = joint_transform(image=image, mask=label)
    aug_image = joint["image"]
    aug_label = joint["mask"]
    aug_image = image_only_transform(image=aug_image)["image"]

    aug_sample_id = f"{sample_id}_aug"
    aug_out_dir = os.path.join(OUTPUT_DIR, aug_sample_id)
    os.makedirs(aug_out_dir, exist_ok=True)
    Image.fromarray(aug_image).save(os.path.join(aug_out_dir, "rgb.jpg"))
    Image.fromarray(aug_label.astype("uint8")).save(os.path.join(aug_out_dir, "labels.png"))
    all_sample_ids.append(aug_sample_id)

# Scrivi il nuovo file .txt
with open(NEW_TXT_PATH, 'w') as f:
    for sample_id in all_sample_ids:
        f.write(sample_id + "\n")

print(f"✅ Completato! Salvato nuovo file: {NEW_TXT_PATH}")

C:\Users\sasyc\anaconda3\envs\mygpuenv\lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
Saving original + selective augment: 100%|██████████| 670/670 [00:23<00:00, 28.51it/s]

✅ Completato! Salvato nuovo file: /Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_augmented456_fold5.txt


In [11]:
#ANALISI DATASET, RESTITUISCE UN DATA
import sys
import pandas as pd

# Importa la funzione di analisi
from utils.analysis_utils import analyze_class_distribution

# Specifica il percorso del dataset
DATASET_DIR = '/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/augmented456_train_fold5'

# Esegui l'analisi
df = analyze_class_distribution(DATASET_DIR)

# Mostra il DataFrame in forma tabellare
df.style.set_caption("Distribuzione delle Classi").format(precision=2)


Analyzing: 100%|██████████| 1014/1014 [00:19<00:00, 53.25it/s]


,Class,Pixel %,Presence % (images)
0,0,5.98,99.90
1,1,15.45,60.65
2,2,11.78,64.99
3,3,14.34,55.72
4,4,0.25,6.11
5,5,1.11,23.37
6,6,7.88,50.39
7,7,33.66,99.31
8,8,9.55,90.63


In [ ]:
import os
import random
import gc
import json
import numpy as np
import torch

from torch.utils.data import DataLoader

# Imposta seed per riproducibilità
SEED = 14
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



from torch.utils.data import DataLoader
from utils.dataset import SegmentationDataset
from utils.model import DeepLabV3PlusWithMobileNet
from utils.losses import get_weighted_crossentropy_loss, combined_loss
from utils.metrics import compute_miou, compute_pixel_accuracy

# CONFIG
BATCH_SIZE = 4
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 9
EPOCHS = 50
ALPHA = 0.7
LR = 1e-4
MIN_DELTA = 1e-4
PATIENCE = 10
MAX_MEMORY_MB = 5120  # 5 GB

RESULTS = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for fold in [5]:
    print(f"\n Inizio Fold {fold}\n{'-'*40}")

    train_dataset = SegmentationDataset(
        root_dir="/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/augmented456_train_fold5",
        id_list_file=f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_augmented456_fold{fold}.txt",
        image_size=IMAGE_SIZE,
    )
    val_dataset = SegmentationDataset(
        root_dir="/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset",
        id_list_file=f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/val/val_fold{fold}.txt",
        image_size=IMAGE_SIZE
    )

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    model = DeepLabV3PlusWithMobileNet(num_classes=NUM_CLASSES).to(device)
    ce_loss_fn = get_weighted_crossentropy_loss(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5
    )

    best_miou = -1.0
    best_val_loss = float('inf')
    epochs_no_improve = 0
    fold_results = []

    for epoch in range(1, EPOCHS + 1):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = combined_loss(outputs, labels, ce_loss_fn, alpha=ALPHA)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        total_miou = 0.0
        total_acc = 0.0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = combined_loss(outputs, labels, ce_loss_fn, alpha=ALPHA)
                val_loss += loss.item()
                total_miou += compute_miou(outputs, labels, num_classes=NUM_CLASSES, ignore_index=0)
                total_acc += compute_pixel_accuracy(outputs, labels, ignore_index=0)

        val_loss /= len(val_loader)
        avg_miou = total_miou / len(val_loader)
        scheduler.step(val_loss)
        avg_acc = total_acc / len(val_loader)

        fold_results.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "mIoU": avg_miou,
            "accuracy": avg_acc
        })

        print(f"[Fold {fold} | Epoch {epoch:02d}] "
              f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
              f"mIoU: {avg_miou:.4f} | Acc: {avg_acc:.4f}")

        if avg_miou > best_miou:
            best_miou = avg_miou
            epochs_no_improve = 0
            model_path = f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/best_model_fold{fold}.pth"
            torch.save(model.state_dict(), model_path)
            print(f"Salvataggio modello Fold {fold} | Nuova mIoU: {best_miou:.4f}")

        if epoch == 1 or val_loss < best_val_loss - MIN_DELTA:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"EARLY STOPPING FOLD {fold}: no val_loss improvement for {PATIENCE} epochs.")
            break


        if device.type == 'cuda':
            mem_allocated = torch.cuda.memory_allocated(device) / (1024 ** 2)
            if mem_allocated > MAX_MEMORY_MB:
                print(f"GPU MEMORY LIMIT EXCEEDED: {mem_allocated:.2f} MB")
                break

        torch.cuda.empty_cache()
        gc.collect()

    RESULTS.append({
        "fold": fold,
        "best_miou": best_miou,
        "epoch_results": fold_results
    })

# ✅ Salva i risultati totali in un file
import json
with open("/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_results.json", "w") as f:
    json.dump(RESULTS, f, indent=2)

print("\nCross-validation completata. Tutti i modelli e risultati salvati.")


 Inizio Fold 5
----------------------------------------
[Fold 5 | Epoch 01] Train Loss: 1.1204 | Val Loss: 0.8438 | mIoU: 0.4238 | Acc: 0.7474
Salvataggio modello Fold 5 | Nuova mIoU: 0.4238
[Fold 5 | Epoch 02] Train Loss: 0.8475 | Val Loss: 0.6918 | mIoU: 0.5183 | Acc: 0.8159
Salvataggio modello Fold 5 | Nuova mIoU: 0.5183
[Fold 5 | Epoch 03] Train Loss: 0.7171 | Val Loss: 0.6011 | mIoU: 0.5153 | Acc: 0.8103
[Fold 5 | Epoch 04] Train Loss: 0.6309 | Val Loss: 0.5660 | mIoU: 0.5087 | Acc: 0.8076
[Fold 5 | Epoch 05] Train Loss: 0.5712 | Val Loss: 0.5420 | mIoU: 0.5189 | Acc: 0.8042
Salvataggio modello Fold 5 | Nuova mIoU: 0.5189
[Fold 5 | Epoch 06] Train Loss: 0.5196 | Val Loss: 0.5305 | mIoU: 0.5369 | Acc: 0.8114
Salvataggio modello Fold 5 | Nuova mIoU: 0.5369
[Fold 5 | Epoch 07] Train Loss: 0.4857 | Val Loss: 0.5280 | mIoU: 0.5283 | Acc: 0.8021
[Fold 5 | Epoch 08] Train Loss: 0.4326 | Val Loss: 0.4848 | mIoU: 0.5565 | Acc: 0.8292
Salvataggio modello Fold 5 | Nuova mIoU: 0.5565
[Fold 5

In [2]:
import os
import albumentations as A
from tqdm import tqdm
from PIL import Image
import numpy as np
from utils.color_utils import save_indexed_label_with_palette

# --- Bounding box con margine ---
def get_bbox_with_margin(mask, target_class, margin=0.25):
    ys, xs = np.where(mask == target_class)
    if len(xs) == 0 or len(ys) == 0:
        return None
    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()

    h, w = mask.shape
    bw = x_max - x_min
    bh = y_max - y_min
    if bw < 30 or bh < 30:
        return None

    pad_x = int(bw * margin)
    pad_y = int(bh * margin)

    x_min = max(0, x_min - pad_x)
    x_max = min(w, x_max + pad_x)
    y_min = max(0, y_min - pad_y)
    y_max = min(h, y_max + pad_y)

    return x_min, y_min, x_max, y_max

# --- Augmentation per immagine originale ---
def augment_original_image(image):
    aug = A.OneOf([
        A.Sharpen(alpha=(0.1, 0.3), lightness=(0.9, 1.0), kernel_size=7, p=1),
        A.ColorJitter(brightness=[0.8, 1.2], contrast=[0.8, 1.2], saturation=[0.8, 1.2], hue=0.05, p=1),
        A.CLAHE(clip_limit=7.0, tile_grid_size=(12,12), p=1),
        A.RandomBrightnessContrast(brightness_limit=[0.2, 0.2], contrast_limit=[-0.2, 0.2], p=1),
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=10, p=1),
    ], p=1.0)
    return aug(image=image)["image"]

# --- CLAHE su immagine con probabilità ---
def apply_clahe_with_probability(image, prob=0.7):
    if np.random.rand() < prob:
        return A.CLAHE(clip_limit=7.0, tile_grid_size=(12, 12), p=1)(image=image)["image"]
    return image

# --- Funzione di zoom + flip ---
def zoom_and_flip(image, label, cls_id, sample_id, output_dir):
    bbox = get_bbox_with_margin(label, cls_id)
    if bbox is None:
        return []

    x_min, y_min, x_max, y_max = bbox
    cropped_img = image[y_min:y_max, x_min:x_max]
    cropped_lbl = label[y_min:y_max, x_min:x_max]

    resize = A.Resize(320, 320, interpolation=Image.BILINEAR)
    resized_img = resize(image=cropped_img)["image"]
    resized_lbl = A.Resize(320, 320, interpolation=Image.NEAREST)(image=cropped_lbl)["image"]

    saved_ids = []

    # Zoomato normale
    zoom_id = f"{sample_id}_zoom{cls_id}"
    zoom_dir = os.path.join(output_dir, zoom_id)
    os.makedirs(zoom_dir, exist_ok=True)
    Image.fromarray(resized_img).save(os.path.join(zoom_dir, "rgb.jpg"))
    save_indexed_label_with_palette(resized_lbl.astype("uint8"), os.path.join(zoom_dir, "labels.png"))
    saved_ids.append(zoom_id)

    # CLAHE sullo zoomato (probabilmente)
    clahe_img = apply_clahe_with_probability(resized_img)
    clahe_id = f"{zoom_id}_clahe"
    clahe_dir = os.path.join(output_dir, clahe_id)
    os.makedirs(clahe_dir, exist_ok=True)
    Image.fromarray(clahe_img).save(os.path.join(clahe_dir, "rgb.jpg"))
    save_indexed_label_with_palette(resized_lbl.astype("uint8"), os.path.join(clahe_dir, "labels.png"))
    saved_ids.append(clahe_id)

    # Flip
    flip = A.HorizontalFlip(p=1.0)
    flipped_img = flip(image=resized_img)["image"]
    flipped_lbl = flip(image=resized_lbl)["image"]

    flip_id = f"{zoom_id}_flip"
    flip_dir = os.path.join(output_dir, flip_id)
    os.makedirs(flip_dir, exist_ok=True)
    Image.fromarray(flipped_img).save(os.path.join(flip_dir, "rgb.jpg"))
    save_indexed_label_with_palette(flipped_lbl.astype("uint8"), os.path.join(flip_dir, "labels.png"))
    saved_ids.append(flip_id)

    # CLAHE sul flipped
    clahe_flipped_img = apply_clahe_with_probability(flipped_img)
    clahe_flip_id = f"{flip_id}_clahe"
    clahe_flip_dir = os.path.join(output_dir, clahe_flip_id)
    os.makedirs(clahe_flip_dir, exist_ok=True)
    Image.fromarray(clahe_flipped_img).save(os.path.join(clahe_flip_dir, "rgb.jpg"))
    save_indexed_label_with_palette(flipped_lbl.astype("uint8"), os.path.join(clahe_flip_dir, "labels.png"))
    saved_ids.append(clahe_flip_id)

    return saved_ids

# --- Percorsi ---
DATASET_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset"
TXT_PATH = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/train_fold5.txt"
OUTPUT_DIR = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/augmented_zoom_flip_clahe_fold5"
NEW_TXT_PATH = "/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/augmented_zoom_flip_clahe_fold5.txt"

os.makedirs(OUTPUT_DIR, exist_ok=True)

ZOOM_CLASSES = [4, 5]
all_sample_ids = []

with open(TXT_PATH, 'r') as f:
    sample_ids = [line.strip() for line in f.readlines()]

for sample_id in tqdm(sample_ids, desc="Augment + Zoom + Flip + CLAHE"):
    input_dir = os.path.join(DATASET_DIR, sample_id)
    img_path = os.path.join(input_dir, "rgb.jpg")
    lbl_path = os.path.join(input_dir, "labels.png")

    if not os.path.exists(img_path) or not os.path.exists(lbl_path):
        print(f"⚠ File mancante: {sample_id}")
        continue

    image = np.array(Image.open(img_path).convert("RGB"))
    label = np.array(Image.open(lbl_path))

    # Originale
    orig_dir = os.path.join(OUTPUT_DIR, sample_id)
    os.makedirs(orig_dir, exist_ok=True)
    Image.fromarray(image).save(os.path.join(orig_dir, "rgb.jpg"))
    save_indexed_label_with_palette(label.astype("uint8"), os.path.join(orig_dir, "labels.png"))
    all_sample_ids.append(sample_id)

    # Versione augmentata
    aug_image = augment_original_image(image)
    aug_id = f"{sample_id}_aug"
    aug_dir = os.path.join(OUTPUT_DIR, aug_id)
    os.makedirs(aug_dir, exist_ok=True)
    Image.fromarray(aug_image).save(os.path.join(aug_dir, "rgb.jpg"))
    save_indexed_label_with_palette(label.astype("uint8"), os.path.join(aug_dir, "labels.png"))
    all_sample_ids.append(aug_id)

    # Zoom + flip + CLAHE su classi 4 e 5
    for cls_id in ZOOM_CLASSES:
        new_ids = zoom_and_flip(image, label, cls_id, sample_id, OUTPUT_DIR)
        all_sample_ids.extend(new_ids)

# --- Salva nuovo TXT ---
with open(NEW_TXT_PATH, 'w') as f:
    for sid in all_sample_ids:
        f.write(sid + "\n")

print(f"✅ Completato. Salvati {len(all_sample_ids)} campioni in: {NEW_TXT_PATH}")

Augment + Zoom + Flip + CLAHE: 100%|██████████| 670/670 [00:46<00:00, 14.39it/s]

✅ Completato. Salvati 1792 campioni in: /Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/augmented_zoom_flip_clahe_fold5.txt


In [3]:
import os
import random
import gc
import json
import numpy as np
import torch

from torch.utils.data import DataLoader

# Imposta seed per riproducibilità
SEED = 14
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



from torch.utils.data import DataLoader
from utils.dataset import SegmentationDataset
from utils.model import DeepLabV3PlusWithMobileNet
from utils.losses import get_weighted_crossentropy_loss, combined_loss
from utils.metrics import compute_miou, compute_pixel_accuracy

# CONFIG
BATCH_SIZE = 4
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 9
EPOCHS = 50
ALPHA = 0.7
LR = 1e-4
MIN_DELTA = 1e-4
PATIENCE = 10
MAX_MEMORY_MB = 5120  # 5 GB

RESULTS = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for fold in [5]:
    print(f"\n Inizio Fold {fold}\n{'-'*40}")

    train_dataset = SegmentationDataset(
        root_dir="/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/augmented_zoom_flip_clahe_fold5",
        id_list_file=f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/train/augmented_zoom_flip_clahe_fold{fold}.txt",
        image_size=IMAGE_SIZE,
    )
    val_dataset = SegmentationDataset(
        root_dir="/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/dataset/dataset",
        id_list_file=f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_splits/val/val_fold{fold}.txt",
        image_size=IMAGE_SIZE
    )

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    model = DeepLabV3PlusWithMobileNet(num_classes=NUM_CLASSES).to(device)
    ce_loss_fn = get_weighted_crossentropy_loss(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5
    )

    best_miou = -1.0
    best_val_loss = float('inf')
    epochs_no_improve = 0
    fold_results = []

    for epoch in range(1, EPOCHS + 1):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = combined_loss(outputs, labels, ce_loss_fn, alpha=ALPHA)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        total_miou = 0.0
        total_acc = 0.0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = combined_loss(outputs, labels, ce_loss_fn, alpha=ALPHA)
                val_loss += loss.item()
                total_miou += compute_miou(outputs, labels, num_classes=NUM_CLASSES, ignore_index=0)
                total_acc += compute_pixel_accuracy(outputs, labels, ignore_index=0)

        val_loss /= len(val_loader)
        avg_miou = total_miou / len(val_loader)
        scheduler.step(val_loss)
        avg_acc = total_acc / len(val_loader)

        fold_results.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "mIoU": avg_miou,
            "accuracy": avg_acc
        })

        print(f"[Fold {fold} | Epoch {epoch:02d}] "
              f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
              f"mIoU: {avg_miou:.4f} | Acc: {avg_acc:.4f}")

        if avg_miou > best_miou:
            best_miou = avg_miou
            epochs_no_improve = 0
            model_path = f"/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/best_model_fold{fold}.pth"
            torch.save(model.state_dict(), model_path)
            print(f"Salvataggio modello Fold {fold} | Nuova mIoU: {best_miou:.4f}")

        if epoch == 1 or val_loss < best_val_loss - MIN_DELTA:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"EARLY STOPPING FOLD {fold}: no val_loss improvement for {PATIENCE} epochs.")
            break


        if device.type == 'cuda':
            mem_allocated = torch.cuda.memory_allocated(device) / (1024 ** 2)
            if mem_allocated > MAX_MEMORY_MB:
                print(f"GPU MEMORY LIMIT EXCEEDED: {mem_allocated:.2f} MB")
                break

        torch.cuda.empty_cache()
        gc.collect()

    RESULTS.append({
        "fold": fold,
        "best_miou": best_miou,
        "epoch_results": fold_results
    })

# ✅ Salva i risultati totali in un file
import json
with open("/Users/sasyc/Downloads/ML Project Work/2025_ml_gr14/2025_ml_gr14/kfold_results.json", "w") as f:
    json.dump(RESULTS, f, indent=2)

print("\nCross-validation completata. Tutti i modelli e risultati salvati.")


 Inizio Fold 5
----------------------------------------
[Fold 5 | Epoch 01] Train Loss: 1.0728 | Val Loss: 0.7424 | mIoU: 0.4827 | Acc: 0.7827
Salvataggio modello Fold 5 | Nuova mIoU: 0.4827
[Fold 5 | Epoch 02] Train Loss: 0.7765 | Val Loss: 0.5820 | mIoU: 0.5111 | Acc: 0.8095
Salvataggio modello Fold 5 | Nuova mIoU: 0.5111
[Fold 5 | Epoch 03] Train Loss: 0.6575 | Val Loss: 0.5599 | mIoU: 0.5142 | Acc: 0.8054
Salvataggio modello Fold 5 | Nuova mIoU: 0.5142
[Fold 5 | Epoch 04] Train Loss: 0.5773 | Val Loss: 0.5318 | mIoU: 0.4921 | Acc: 0.7830
[Fold 5 | Epoch 05] Train Loss: 0.5164 | Val Loss: 0.4980 | mIoU: 0.5081 | Acc: 0.8026
[Fold 5 | Epoch 06] Train Loss: 0.4601 | Val Loss: 0.4904 | mIoU: 0.5379 | Acc: 0.8172
Salvataggio modello Fold 5 | Nuova mIoU: 0.5379
[Fold 5 | Epoch 07] Train Loss: 0.4146 | Val Loss: 0.4621 | mIoU: 0.5526 | Acc: 0.8319
Salvataggio modello Fold 5 | Nuova mIoU: 0.5526
[Fold 5 | Epoch 08] Train Loss: 0.3641 | Val Loss: 0.4650 | mIoU: 0.5455 | Acc: 0.8298
[Fold 5